In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import jax.numpy as jnp
from jax import grad, vmap
import jax
import jax.random as jrandom
import matplotlib.pyplot as plt
from utils import prepare_data
from models import *
from advi import *
import numpy as np 

## Faces Experiments 

In [3]:
from scipy.io import loadmat

all_data = loadmat("data/frey_rawface.mat")
data = all_data["ff"].T

In [4]:
rank = 10
nmf_model = NMF_Model_PoissonGamma(data, rank, gamma_prior_shape=1, gamma_prior_scale=1)
nmf_dim = nmf_model.dim
k = jrandom.key(15)
trial_vec = jrandom.normal(k, shape=(nmf_dim,))
theta, beta = nmf_model.t_inv_map(trial_vec)
nmf_advi = mean_field_advi(nmf_model)




An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [5]:
nmf_advi = mean_field_advi(nmf_model)
nmf_advi.run_advi(k, 10, 5001, 1e-6, print_every=500, adaptive=True)


-51112680.0
-51441292.0
-50973064.0
-49775732.0
-49645380.0
-50717588.0
-50330264.0
-49903490.0
-49688304.0
-48279040.0


In [ ]:
thetas, betas = nmf_model.t_inv_map(nmf_advi.params["mu"])

u = 42
fig, axs = plt.subplots(2, 5, figsize=(12, 5), constrained_layout=True)
axs = axs.flatten()
for i in range(10):
    axs[i].imshow(betas[i].reshape(28, 20), cmap="gray")
    axs[i].axis("off")
    # axs[i].set_title(rf"$\beta_{{{i}}}$")
    
plt.savefig("face_poisson gamma.pdf")

In [ ]:
rank = 10
all_data = loadmat("data/frey_rawface.mat")
data = all_data["ff"].T
nmf_model = old_NMF_Model_PoissonDirExp(data, rank,dir_prior=100, exp_prior=0.1)
nmf_dim = nmf_model.dim
k = jrandom.key(15)
trial_vec = jrandom.normal(k, shape=(nmf_dim,))
theta, beta = nmf_model.t_inv_map(trial_vec)
nmf_advi = mean_field_advi(nmf_model)
nmf_advi.run_advi(k, 10, 5001, 1e-5, print_every=500, adaptive=True)


thetas, betas = nmf_model.t_inv_map(nmf_advi.params["mu"])

u = 42
fig, axs = plt.subplots(2, 5, figsize=(12, 5))
axs = axs.flatten()
for i in range(10):
    axs[i].imshow(betas[i].reshape(28, 20), cmap="gray")
    axs[i].axis("off")
    # axs[i].set_title(rf"$\beta_{{{i}}}$")

In [ ]:
nmf_advi.run_advi(k, 10, 5001, 1e-5, print_every=500, adaptive=True)


In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(12, 5))
axs = axs.flatten()
for i in range(10):
    axs[i].imshow(betas[i].reshape(28, 20), cmap="gray")
    axs[i].axis("off")
    # axs[i].set_title(rf"$\beta_{{{i}}}$")

In [ ]:

#Exploring reconstructions 
u = 25
plt.figure()
plt.subplot(121)
plt.imshow((thetas[u] @ betas).reshape(28, 20), cmap="gray")
plt.axis("off")
plt.title(rf"$\theta_{{{u}}}\cdot\beta$")
plt.subplot(122)
plt.imshow((data[u]).reshape(28, 20), cmap="gray")
plt.title(rf"$Y_{{{u}, true}}$")
plt.axis("off")
print(f"theta_{u}: {thetas[u]}")